In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import matplotlib.pyplot as plt
import numpy as np

# Import the API key.
from config import g_key

# Import the API key.
from config import weather_api_key

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [2]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
0,Hobart,AU,2020-01-05 08:38:14,-42.88,147.33,64.40,45,20,10.29,few clouds,0.00,0.0
1,Naze,NG,2020-01-05 08:41:41,5.43,7.07,73.40,35,11,10.29,dust,0.00,0.0
2,Kabare,RW,2020-01-05 08:41:41,-1.65,29.38,62.02,84,81,3.78,light rain,2.38,0.0
3,Ushuaia,AR,2020-01-05 08:38:14,-54.81,-68.31,50.00,66,40,10.29,scattered clouds,0.00,0.0
4,Amahai,ID,2020-01-05 08:41:42,-3.31,129.00,85.62,67,36,4.38,light rain,1.81,0.0


In [4]:
# Verify lat and lng are float64
city_data_df.dtypes

City                         object
Country                      object
Date                         object
Lat                         float64
Lng                         float64
Max Temp                    float64
Humidity                      int64
Cloudiness                    int64
Wind Speed                  float64
Current Description          object
Rain Inches (last 3 hrs)    float64
Snow Inches (last 3 hrs)    float64
dtype: object

In [5]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_liketh = input("Do you want it to be raining? (yes/no) ")
snow_liketh = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) yes
Do you want it to be snowing? (yes/no) no


In [6]:
# Convert rain/snow responses to boolean
rain_liketh_bool = False
snow_liketh_bool = False

if rain_liketh == "yes":
    rain_liketh_bool = True
else:
    rain_liketh_bool = False

if snow_liketh == "yes":
    snow_liketh_bool = True
else:
    snow_liketh_bool = False

In [7]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain Inches (last 3 hrs)"].astype('bool') == rain_liketh_bool) & \
                                       (city_data_df["Snow Inches (last 3 hrs)"].astype('bool') == snow_liketh_bool)]
preferred_cities_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
4,Amahai,ID,2020-01-05 08:41:42,-3.31,129.00,85.62,67,36,4.38,light rain,1.81,0.0
6,Cayenne,GF,2020-01-05 08:41:42,4.94,-52.33,77.00,88,19,3.36,light rain,0.44,0.0
10,Avarua,CK,2020-01-05 08:41:43,-21.21,-159.78,77.00,94,100,1.12,light rain,0.25,0.0
30,Inyonga,TZ,2020-01-05 08:41:47,-6.72,32.06,79.90,65,100,3.18,light rain,1.44,0.0
39,Butaritari,KI,2020-01-05 08:38:27,3.07,172.79,84.99,78,100,12.75,light rain,0.56,0.0
45,Atuona,PF,2020-01-05 08:38:42,-9.80,-139.03,79.95,74,0,6.82,light rain,0.25,0.0
108,Dingle,PH,2020-01-05 08:42:03,11.00,122.67,77.54,87,12,13.18,light rain,0.56,0.0
145,Sao Jose Da Coroa Grande,BR,2020-01-05 08:42:11,-8.90,-35.15,79.52,76,93,11.39,light rain,0.13,0.0
153,Sambava,MG,2020-01-05 08:42:12,-14.27,50.17,84.78,75,100,3.83,light rain,2.44,0.0
169,Faanui,PF,2020-01-05 08:38:17,-16.48,-151.75,81.39,78,87,11.79,light rain,0.44,0.0


In [8]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [9]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Current Description", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

In [10]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.


In [11]:
# Verify hotel dataframe has hotels with city information
hotel_df.head()

,City,Country,Current Description,Max Temp,Lat,Lng,Hotel Name
4,Amahai,ID,light rain,85.62,-3.31,129.00,Penginapan Madina
6,Cayenne,GF,light rain,77.00,4.94,-52.33,Hôtel Le Dronmi
10,Avarua,CK,light rain,77.00,-21.21,-159.78,Paradise Inn
30,Inyonga,TZ,light rain,79.90,-6.72,32.06,Eden Rock Lodge
39,Butaritari,KI,light rain,84.99,3.07,172.79,Isles Sunset Lodge


In [12]:
# Info box data
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [17]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
cur_desc = hotel_df["Current Description"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Save map
# plt.savefig("weather_data\WeatherPy_vacation_map.png")

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Create the output file (CSV).
output_data_file = "data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index=False)